Install Requirements

In [ ]:
pip install pandas

Import Libraries

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.models import alexnet
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

Set Arguments

In [ ]:
NUM_CLASSES = 101 + 1

BATCH_SIZE = 256

LR = 1e-3
MOMENTUM = 0.9
WEIGHT_DECAY = 5e-5

NUM_EPOCHS = 30
STEP_SIZE = 20
GAMMA = .1

Define Data Preprocessing

In [ ]:
from torchvision import transforms
mean = (.5, .5, .5)
std = (.5, .5, .5)
transform = transforms.Compose([transforms.Resize(256),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize(mean = mean, std = std)])

eval_transform =transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


Prepare the Data Class

In [ ]:
!git clone "https://github.com/MachineLearning2020/Homework2-Caltech101.git"

Cloning into 'Homework2-Caltech101'...
remote: Enumerating objects: 9256, done.
remote: Total 9256 (delta 0), reused 0 (delta 0), pack-reused 9256
Receiving objects: 100% (9256/9256), 129.48 MiB | 27.70 MiB/s, done.
Resolving deltas: 100% (4/4), done.
Updating files: 100% (9149/9149), done.


In [ ]:
# this way you could run it without adding it to the notebook
!python caltech_dataset.py

In [ ]:
from torchvision import transforms
split = 'train'

file = open("/content/Homework2-Caltech101/"+ split + ".txt", 'r')
lines = file.readlines()
print(lines[100].split("/")[0])
path = "/content/Homework2-Caltech101/101_ObjectCategories/" + lines[100]
image = pil_loader(path.strip())
transform = transforms.ToTensor()
tensor = transform(image)
tensor.shape



airplanes


torch.Size([3, 196, 394])

In [ ]:
from torchvision.datasets import VisionDataset
from PIL import Image

import os
import os.path
import sys

def pil_loader(path):
  with open(path, 'rb') as f:
    img = Image.open(f)
    return img.convert('RGB')

class Caltech(VisionDataset):
  def __init__(self, root, split = 'train', transform = None, target_transform = None):
    super(Caltech, self).__init__(root, transform = transform, target_transform=target_transform)
    self.split = split
    self.root = root
    self.transform = transform
    self.image_dict = {}
    self.class_dict = {}

    split = 'train'

    file = open("/content/Homework2-Caltech101/"+ split + ".txt", 'r')
    lines = file.readlines()
    index = 0
    class_index = 0
    for line in lines:
      path = self.root + line
      label = line.split("/")[0]
      if label not in self.class_dict.keys():
        self.class_dict[label] = class_index
        self.image_dict[index] = (pil_loader(path.strip()),class_index)
        class_index += 1
      else:
        target_class = self.class_dict[label]
        self.image_dict[index] = (pil_loader(path.strip()),target_class)

      index += 1


  def __getitem__(self, index):
    image, label = self.image_dict[index]
    if self.transform is not None:
      image = self.transform(image)
    return image, label
  def __len__(self):
    return len(self.image_dict)

In [ ]:
import pathlib
print(pathlib.Path().resolve())
if not os.path.isdir('./Caltech101'):
  !git clone https://github.com/MachineLearning2020/Homework2-Caltech101.git

DATA_DIR = "/content/Homework2-Caltech101/101_ObjectCategories/"

training_data = Caltech(DATA_DIR, 'train', transform = transform)
test_data = Caltech(DATA_DIR, 'test', transform = eval_transform)


/content/Homework2-Caltech101
fatal: destination path 'Homework2-Caltech101' already exists and is not an empty directory.


In [ ]:
train_loader = DataLoader(training_data, batch_size = BATCH_SIZE, shuffle = True, drop_last=True)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE )

Preparing the Network

In [ ]:
net = alexnet()
net.classifier[6] = nn.Linear(4096, NUM_CLASSES)

Prepare for Training


In [ ]:
criterion = nn.CrossEntropyLoss()
parameters_to_optimize = net.parameters()
optimizer = optim.SGD(parameters_to_optimize, lr = LR , momentum = MOMENTUM, weight_decay= WEIGHT_DECAY )

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = STEP_SIZE, gamma = GAMMA)
print(len(train_loader))

23


In [ ]:
current_step = 0
for epoch in range(NUM_EPOCHS):
  for images, labels in train_loader:
    net.train()
    optimizer.zero_grad()
    outputs = net(images)
    loss = criterion(outputs, labels)
    if current_step % 10 == 0:
      print("loss of step: ", current_step,"is: ", loss )
    current_step += 1
    loss.backward()
    optimizer.step()

  scheduler.step()



loss of step:  0 is:  tensor(4.6245, grad_fn=<NllLossBackward0>)
loss of step:  10 is:  tensor(4.6232, grad_fn=<NllLossBackward0>)
loss of step:  20 is:  tensor(4.6214, grad_fn=<NllLossBackward0>)
loss of step:  30 is:  tensor(4.6198, grad_fn=<NllLossBackward0>)
loss of step:  40 is:  tensor(4.6161, grad_fn=<NllLossBackward0>)
loss of step:  50 is:  tensor(4.6150, grad_fn=<NllLossBackward0>)
loss of step:  60 is:  tensor(4.6126, grad_fn=<NllLossBackward0>)
loss of step:  70 is:  tensor(4.6036, grad_fn=<NllLossBackward0>)
loss of step:  80 is:  tensor(4.6040, grad_fn=<NllLossBackward0>)
loss of step:  90 is:  tensor(4.5993, grad_fn=<NllLossBackward0>)
loss of step:  100 is:  tensor(4.5961, grad_fn=<NllLossBackward0>)
loss of step:  110 is:  tensor(4.5988, grad_fn=<NllLossBackward0>)
loss of step:  120 is:  tensor(4.5956, grad_fn=<NllLossBackward0>)
loss of step:  130 is:  tensor(4.5909, grad_fn=<NllLossBackward0>)
